# Verkehrsfluss

Die Stadt New York teilt ausführliche Verkehrsinformationen [hier](https://data.cityofnewyork.us/Transportation/Real-Time-Traffic-Speed-Data/qkm5-nuaq).
Diese Verkehrsdaten möchten wir dahingehend untersuchen, ob sich für bestimmte Straßenabschnitte wiederkehrende Muster ergeben.

In [2]:
import os
import random

import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

pd.options.display.max_rows = 10  # zeige maximal 10 Einträge eines DataFrames an

Als Erstes werden die Positionen der Streckenabschnitte geladen.
Eine ausführliche Erklärung, was die Attribute bedeuten, ist [hier](https://dev.socrata.com/foundry/data.cityofnewyork.us/i4gi-tjb9) zu finden.

In [15]:
df = pd.read_json("https://data.cityofnewyork.us/resource/i4gi-tjb9.json")
df.head()

,borough,data_as_of,encoded_poly_line,encoded_poly_line_lvls,id,link_id,link_name,link_points,owner,speed,status,transcom_id,travel_time
0,Bronx,2019-10-14T04:04:03.000,gsbxFfftaMrFQ`BQvBWxBs@rCcBpe@wVnb@iU,BBBBBBBB,168,4456496,BWB S Toll Plaza - Queens Anchorage,"40.814761,-73.83668 40.81354,-73.83659 40.8130...",NYC_DOT_LIC,52.81,0,4456496,82
1,Brooklyn,2019-10-14T04:04:03.000,gnyvF~i~bMcCsKaPys@wBuJo@yC,BBBBB,411,4763652,VNB E SI GANTRY UPPER LEVEL - BROOKLYN GANTRY ...,"40.6040405,-74.052321 40.6047,-74.050301 40.60...",Verrazano-Narrows-Bridge,55.30,0,4763652,83
2,Queens,2019-10-14T04:04:03.000,}uvwFlebaMkMcUcIkNoH}N}AeEaAaEqC_KqAuDaCsE_EgH...,BBBBBBBBBBBBBB,319,4362251,LIE WB LITTLE NECK PKWY - NB CVE,"40.7537504,-73.744391 40.75605,-73.740851 40.7...",NYC-DOT-Region 10,57.78,0,4362251,166
3,Queens,2019-10-14T04:04:03.000,cl}wFtbkaMfFiIdByBrD{DtAeAj@_@nA}@\\\\|BcAvBy@...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...,212,4362244,CVE NB LIE - WILLETS PT BLVD,"40.78802,-73.79003 40.7868604,-73.78838 40.786...",NYC-DOT-Region 10,52.81,0,4362244,206
4,Manhattan,2019-10-14T04:04:03.000,gkhwFp~tbMoJdGkHnCaK`Fat@d`@oe@`NyD\\\\|@{L@wH...,BBBBBBBBBB,124,4456501,BBT W Toll Plaza - Manhattan Portal,"40.68036,-74.00441001 40.6822,-74.0057201 40.6...",MTA Bridges & Tunnels,47.84,-101,4456501,148


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
borough                   1000 non-null object
data_as_of                1000 non-null object
encoded_poly_line         1000 non-null object
encoded_poly_line_lvls    1000 non-null object
id                        1000 non-null int64
link_id                   1000 non-null int64
link_name                 1000 non-null object
link_points               1000 non-null object
owner                     1000 non-null object
speed                     1000 non-null float64
status                    1000 non-null int64
transcom_id               1000 non-null int64
travel_time               1000 non-null int64
dtypes: float64(1), int64(5), object(7)
memory usage: 101.6+ KB


In diesem Kontext heißt `object`, dass nur Zeichenketten vorliegen.
`data_as_of` ist dem Namen nach aber keine Zeichenkette, sondern ein Datum.
Somit ist die Repräsentation hier noch nicht ganz optimal.

Aber die `link_id` ist bereits einmalig und die neu erstellte ID von dem Modul `pandas` wird hier nicht benötigt.
Deswegen setzen wir einen neuen Index.

In [16]:
df = df.set_index("link_id")
df.head()

,borough,data_as_of,encoded_poly_line,encoded_poly_line_lvls,id,link_name,link_points,owner,speed,status,transcom_id,travel_time
link_id,,,,,,,,,,,,
4456496,Bronx,2019-10-14T04:04:03.000,gsbxFfftaMrFQ`BQvBWxBs@rCcBpe@wVnb@iU,BBBBBBBB,168,BWB S Toll Plaza - Queens Anchorage,"40.814761,-73.83668 40.81354,-73.83659 40.8130...",NYC_DOT_LIC,52.81,0,4456496,82
4763652,Brooklyn,2019-10-14T04:04:03.000,gnyvF~i~bMcCsKaPys@wBuJo@yC,BBBBB,411,VNB E SI GANTRY UPPER LEVEL - BROOKLYN GANTRY ...,"40.6040405,-74.052321 40.6047,-74.050301 40.60...",Verrazano-Narrows-Bridge,55.30,0,4763652,83
4362251,Queens,2019-10-14T04:04:03.000,}uvwFlebaMkMcUcIkNoH}N}AeEaAaEqC_KqAuDaCsE_EgH...,BBBBBBBBBBBBBB,319,LIE WB LITTLE NECK PKWY - NB CVE,"40.7537504,-73.744391 40.75605,-73.740851 40.7...",NYC-DOT-Region 10,57.78,0,4362251,166
4362244,Queens,2019-10-14T04:04:03.000,cl}wFtbkaMfFiIdByBrD{DtAeAj@_@nA}@\\\\|BcAvBy@...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...,212,CVE NB LIE - WILLETS PT BLVD,"40.78802,-73.79003 40.7868604,-73.78838 40.786...",NYC-DOT-Region 10,52.81,0,4362244,206
4456501,Manhattan,2019-10-14T04:04:03.000,gkhwFp~tbMoJdGkHnCaK`Fat@d`@oe@`NyD\\\\|@{L@wH...,BBBBBBBBBB,124,BBT W Toll Plaza - Manhattan Portal,"40.68036,-74.00441001 40.6822,-74.0057201 40.6...",MTA Bridges & Tunnels,47.84,-101,4456501,148


## Visualisierung der betrachteten Strecke

Zunächst visualisieren wir uns eine spezifische Strecke, die ausgewertet wird.
Dafür wird der Text aus der Tabelle in eine Liste von Listen von Gradzahlen umgerechnet.
`latlon` steht hier für Latitude-Longitude (Längengrad-Breitengrad) Koordinaten.

In [33]:
examined_link_id = 4456501

latlons = []
coordinate_text = df.loc[examined_link_id]["link_points"].iloc[0]
for latlon in coordinate_text.split(" "):
    lat, lon = latlon.split(",")
    lat, lon = float(lat), float(lon)
    latlons.append((lat, lon))

latlons

[(40.68036, -74.00441001),
 (40.6822, -74.0057201),
 (40.6837, -74.00644),
 (40.68563, -74.00757),
 (40.6941205, -74.01288001),
 (40.7002801, -74.01529001),
 (40.70121, -74.0156),
 (40.703434, -74.0156101),
 (40.70499, -74.01519),
 (40.70609, -74.01468)]

Diese kann man nun auf einer Karte darstellen.
Dafür bietet sich `folium` an.

In [37]:
import folium

m = folium.Map(latlons[0], zoom_start=11)
folium.features.PolyLine(latlons, weight=8).add_to(m)
display(m)

In [38]:
#### HIER WEITERMACHEN

### Bereinigen der Daten

Häufig kommt es im CSV-Format zu Duplikaten.
Ebenfalls kommt es manchmal zu Sprüngen, also das Einträge nicht streng der zeitlichen Reihenfolge nach sortiert sind.
Dies ist häufig mit Problemen mit dem technischen System erklärbar.
Solche Fehler wollen wir nun ausfindig machen.
Die Maske zeigt nun mittels FALSE an, wenn es ein einmaliger Eintrag ist und mittels TRUE, wenn es sich um ein Duplikat handelt.

In [ ]:
mask = df.index.duplicated()
mask

Diese Maske kann nun genutzt werden, um die Duplikate sich anzeigen zu lassen.
Wenn ein Eintrag zweimal vorkommt, so gibt es ein Duplikat.
Wenn ein Eintrag $k$ Male vorkommt, so gibt es $k-1$ Duplikate.
Diese lassen wir uns nun anzeigen.

In [ ]:
df.loc[mask]

Nun werden die Duplikate entfernt.
Die Tilde ("~") negiert die Maske, womit nun alle Zeilen ausgewählt werden, die KEINE Duplikate sind.
Mit dieser Auswahl wird die Variable `df` neu belegt.

In [ ]:
df = df.loc[~df.index.duplicated()]

Außerdem muss die richtige zeitliche Reihenfolge sichergestellt sein.
Dies nennt man auch einen monoton ansteigenden Index.

In [ ]:
print("is monotonic increasing: ", df.index.is_monotonic_increasing)
df = df.sort_index()
print("is monotonic increasing: ", df.index.is_monotonic_increasing)

Nun stellt sich die Frage, wie häufig die Daten übermittelt werden.
Mit folgendem Trick bekommt man eine Übersicht:

In [ ]:
pd.DataFrame(data=df.index).diff().describe()

### Auswahl des Attributs

Nun sind die Daten bereinigt. Nun können wir über die `plot`-Methode einzelne Attribute über die Zeit hinweg visualisieren.
Aber eignet sich das Attribut `Speed` oder das Attribut `TravelTime` besser für die Betrachtung des Verkehrs?
Was genau erfassen die Attribute `Speed` und `TravelTime` eigentlich?
Neben den Daten liegt auf der Webseite die [PDF mit den Meta-Daten](http://data.beta.nyc//dataset/e8facf61-2bb1-49e0-9128-5a8797b214c8/resource/6b126292-b251-4e59-bfba-6844a633a3a2/download/metadatatrafficspeeds.pdf).

#### Speed

Im Folgenden werden die Werte geplottet, um einen visuellen Eindruck zu erhalten.
Mit `iloc` werden nur die ersten 300 Einträge geplottet.

In [ ]:
df.Speed.plot()
plt.show()
df.Speed.iloc[:300].plot()
plt.show()

Welche Uhrzeit ist ist an der 300sten Stelle im DataFrame, also dem letzten geplotteten Eintrag?
Nutzen Sie die Methode `.iloc`.

### TravelTime

Vergleichbar mit `Speed` werden auch hier die Werte geplottet.

In [ ]:
df.TravelTime.plot()
plt.show()
df.TravelTime.iloc[0:300].plot()
plt.show()

## Weiterführende Auswertungen

Für sich wiederholende Muster gibt es verschiedene Möglichkeiten, diese zu visualisieren.
Jede Methode hat ihre eigenen Stärken und Schwächen.

### Plots mit Gruppierung

Anstelle der strikten zeitlichen Abfolge können Plots auch anhand verschiedener anderer Kriterien gruppiert und dann ausgewertet werden.
Dies geschieht in pandas mit der Methode `groupby`.

Nun wird die durchschnittliche Geschwindigkeit für jede Stunde des Tages geplottet, also von 00:00 bis 23:00.

In [ ]:
def extract_hour(pandas_timestamp):
    return pandas_timestamp.hour

hourly_speed_mean = df.Speed.groupby(extract_hour).agg('mean')
hourly_speed_mean.plot()
plt.title("Durchschnittliche Geschwindigkeit je Uhrzeit, 0 Uhr bis 23 Uhr")
plt.show()

Nun wird die Standardabweichung dargestellt.
Wie kann man diese nun interpretieren?

In [ ]:
hourly_speed_std = df.Speed.groupby(extract_hour).agg('std')
hourly_speed_std.plot()
plt.show()

## Autokorrelation

Wie vielen Minuten entspricht der Lag von 1? 
Was bedeuten die einzelnen Peaks?
Wie lassen sich die gegebenen Peaks erklären?

Je nachdem, welchen Monat Sie ausgewählt haben, können Sie die Begrenzung auf die ersten $x$ Einträge auch anpassen.
Die hier getroffene Auswahl bezieht sich auf den Juni, in dem ein Peak bei 2000 liegen sollte.

In [ ]:
autocorrelation_plot(df.TravelTime, alpha=0.7)
autocorrelation_plot(df.Speed, alpha=0.7)
plt.xlim([0, 5000])  # Begrenze Plot auf die ersten 5000 Einträge
plt.show()

## Prognose der zweiten Monatshälfte anhand der ersten

Wie lässt sich die zweite Monatshälfte mithilfe der ersten Monatshälfte vorhersagen?
Vermutlich lassen sich hier die Prinzipien des vorherigen Notebooks zum Thema Gezeiten anwenden.

In [ ]:
n = len(df)
df_train, df_test = df.iloc[:(n//2)], df.iloc[(n//2):]
df_train.info()
df_test.info()
df_train

### Einfachste Schätzung: Mittelwert

Eine sehr einfache Annahme ist, dass ein Zufallsprozess vorliegt, der der Gauß'schen Normalverteilung folgt.
Alle Ausschläge werden somit als zufällig angesehen.
Für Normalverteilungen ist der Mittelwert ein guter Schätzer.
Dies ist nun unser Baseline-Modell

In [ ]:
mean = 20  # <-- bestimmen Sie einen passenden Wert
rmse = ((mean - df_test.Speed) ** 2).mean() ** .5
mean, rmse

Jedes Verfahren, das auch nur etwas komplexer als die Mittelwertberechnung ist und dabei höhere Fehler produziert, sollte nur dann verwendet werden, wenn es dafür sehr gute Gründe gibt.
Allgemein gilt, das einfache Mittel bevorzugt werden sollen.

### Naive Schätzung


Der nun vorgestellte naive Schätzer funktioniert vergleichbar mit den Gezeiten, nur dass diesmal nicht für jeden Zeitpunkt ein Eintrag vorliegt.
Deswegen wird eine Hilfsmethode verwendet, die den nächstgelegenen Eintrag zurückgibt.
Wie dies geschieht, braucht Sie nicht weiter interessieren.
Doch wie lange ist ein Zyklus? 
Tragen Sie einen Wert ein, der sich aus der Domäne heraus rechtfertigen lässt.
Schauen Sie gerne auf die Ergebnisse aus der Autokorrelation.

In [ ]:
zyklus_laenge_in_stunden = 30  # <-- bestimmen Sie einen passenden Wert.

def naive_estimator(this_series, t):
    if t < first_day:
        print("Nur für Prognose, nicht in die Vergangenheit")
        return
    nearest_entry = get_nearest_entry(this_series, t)
    if nearest_entry is not None:
        return nearest_entry  # Gebe tatsächlichen Wert zurück
    else:
        t = t - pd.Timedelta(hours=zyklus_laenge_in_stunden)  # Betrachte Wert des vorherigen Zyklus'
        return naive_estimator(this_series, t)

Definieren Sie `first_day` und `last_day` anhand des DataFrames `df_train`, wie es bereits bei den Gezeiten vorgenommen worden ist.

In [ ]:
first_day = pd.Timestamp('2018-06-01 02:58:04')  # <-- tragen Sie hier etwas ein, was unabhängig vom geladenen Monat ist
last_day = pd.Timestamp('2018-06-16 03:08:04')  # <-- tragen Sie hier etwas ein, was unabhängig vom geladenen Monat ist

first_day, last_day

Benutzen Sie nun die Schätzfunktion, um für die zweite Hälfte des Monats den Verkehr zu schätzen.
Am Ende sollte das Ergebnis eine `pd.Series` sein, als Zwischenschritt darf es gerne eine Liste sein.
Es lohnt sich der Blick in das vorherige Notebook.

Plotten Sie zunächst die tatsächlichen Daten.
Plotten Sie danach die geschätzten Werte.
Ein Tipp: Wenn zweimal `.plot()` aufgerufen wird, werden Ihre Ergebnisse automatisch über die bereits geplotteten Daten gelegt.

Der soeben erstellte Plot dürfte etwas unübersichtlich erscheinen.
Deswegen werden wir nun nur die Abweichung von der Vorhersage zu den tatsächlichen Daten, den sogenannten Fehler, betrachten.
Plotten Sie nun den Fehler und ermitteln Sie die Fehler-Statistiken mittels `describe`.
Wird die Geschwindigkeit von der Methode eher über- oder unterschätzt?

Berechnen Sie nun den RMSE dieser Methode